In [1]:
try:
    import cv2
except:
    !pip install opencv-python
    import cv2
import boto3
import numpy as np
import io
import os
from PIL import Image
from botocore.exceptions import ClientError


In [2]:
# Function to download image from S3
def download_image_from_s3(bucket, key):
    s3 = boto3.client('s3')
    response = s3.get_object(Bucket=bucket, Key=key)
    image_data = response['Body'].read()
    image = Image.open(io.BytesIO(image_data))
    image_np = np.array(image)
    return image_np

# Function to upload image to S3
def upload_image_to_s3(image_np, bucket, key):
    s3 = boto3.client('s3')
    _, buffer = cv2.imencode('.jpg', image_np)
    io_buf = io.BytesIO(buffer)
    s3.put_object(Bucket=bucket, Key=key, Body=io_buf.getvalue(), ContentType='image/jpeg')

In [3]:
# Load YOLO model
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")

def process_image(image):
    # Get image dimensions
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    (height, width) = image.shape[:2]

    # Define the neural network input
    blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416), swapRB=False, crop=False)
    net.setInput(blob)

    # Perform forward propagation
    output_layer_name = net.getUnconnectedOutLayersNames()
    output_layers = net.forward(output_layer_name)

    # Initialize list of detected people
    people = []

    # Loop over the output layers
    for output in output_layers:
        # Loop over the detections
        for detection in output:
            # Extract the class ID and confidence of the current detection
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            # Only keep detections with a high confidence
            if class_id == 0 and confidence > 0.80:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                # Add the detection to the list of people
                people.append((x, y, w, h))

    # Draw bounding boxes around the people
    for i, (x, y, w, h) in enumerate(people):
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(image, f'Person {i+1}', (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    return image

In [4]:
def generate_unique_filename(bucket, base_filename, directory):
    counter = 0
    s3 = boto3.client('s3')
    filename, ext = os.path.splitext(base_filename)
    
    while True:
        unique_filename = os.path.join(directory, f"{filename}_{counter}{ext}")
        try:
            s3.head_object(Bucket=bucket, Key=unique_filename)
        except ClientError as e:
            if e.response['Error']['Code'] == '404':
                return unique_filename
            else:
                raise
        counter += 1

def main_run(bucket, key):
    s3 = boto3.client('s3')
    image = download_image_from_s3(bucket, key)
    processed_image = process_image(image)

    # Extract the filename (without extension) from the key
    filename = os.path.splitext(key.split('/')[-1])[0]

    # Generate a unique output key with conflict resolution
    output_key = generate_unique_filename(bucket, filename, 'outputs')

    # Check if people were detected
    if len(processed_image) > 0:  # Check if processed_image contains detections
        try:
            upload_image_to_s3(processed_image, bucket, output_key)
        except ClientError as error:
            raise ClientError(f"Error uploading image to S3: {error}")
        print(f"Processed image saved to: s3://{bucket}/{output_key}")
    else:
        print(f"No people detected in image: {key}")

In [5]:
input_bucket = 'my-image-input-bucket'
input_key = 'inputs/people.jpg'
main_run(input_bucket, input_key)

Processed image saved to: s3://my-image-input-bucket/outputs/people_4
